## Look for low (and high) resolution decisions in a string to check for patterns.
### Use the .tf files from MM's analysis+code. 


In [3]:
# Imports
import pickle
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches
from copy import deepcopy
import plotly.graph_objects as go
from scipy.optimize import curve_fit
from dataclasses import make_dataclass

import sys
module_path = 'code' 
if module_path not in sys.path:
    sys.path.append(module_path)

# Tony's code, from Tony's repo mouse-maze-analysis
from data_loader import DataLoader
from query_data import QueryData

# Markus's code
from MM_Plot_Utils import plot, hist
from MM_Maze_Utils import *
from MM_Traj_Utils import *

%matplotlib inline
%reload_ext autoreload
%autoreload 2


In [4]:
TestTrajModule()

'OK'

In [8]:
# Dictionary of videos
video = {
    # rewarded normal mice
    'B1': '08-08-19_23-55-49_B1_night1',
    'B2': '08-12-19_01-12-41_B2_night',
    'B3': '08-14-19_23-47-14_B3_night1',
    'B4': '08-21-19_01-22-44_B4_night1',
    'C1': '09-16-19_22-27-48_C1',
    'C3': '09-20-19_00-03-55_C3',
    'C6': '09-22-19_23-59-47_C6',
    'C7': '09-23-19_23-14-03_C7',
    'C8': '09-24-19_22-46-27_C8',
    'C9': '09-26-19_22-56-07_C9',
    
    # rewarded homozygous acortical mice
    'L6':'09-29-19_22-55-51_L06',
    'L285':'03-14-20_21-29-20_285actx_default',
    'L286':'03-10-20_21-35-02_286actx_default',
    'Z159':'10-11-20_22-01-23_2159_actx',
}


In [9]:
# Some lists of nicknames for the relevant ones
RewNames=['B1','B2','B3','B4','C1','C3','C6','C7','C8','C9']
homNames=['L6','L285','L286','Z159']
AllNames=RewNames+homNames

In [10]:
work_dir = 'indata' # the subdirectory for raw data files
ma = NewMaze(6) # defines a 6-level maze structure

In [11]:
# Water port location - node 116.
# How many bouts did the animal complete during the experiment?
# How many of these bouts did the animal visit node 116 - independent of the reward delivery?

fn="B1"
tf=LoadTraj(fn+'-tf')
print('Number of bouts for',fn,'is',len(tf.fr[:]))
fwvs =[]
number_of_bouts = len(tf.fr)
for bout in range(number_of_bouts):
    wv= np.where(tf.no[bout][:,0]==116)[0]
    #print(len(wv))
    if len(wv)>0:
        x = (tf.no[bout][:wv[0],0]) #print(tf.no[bout][wv[0]-2:wv[0],0], bout)
        fwvs.append(x)
        
print('Number of bouts with at least one visit to node 116 is',len(fwvs))


Number of bouts for B1 is 104
Number of bouts with at least one visit to node 116 is 67


In [20]:
# Post-observation, look for the low res decision chains
# All 
nickname = 'L286'
res = 31
tf = LoadTraj(nickname+'-tf')
fwvs=[] # all water node visits
bout_fwvs = []
number_of_bouts = len(tf.fr)
for bout in range(number_of_bouts):
    wv= np.where(tf.no[bout][:,0]==116)[0]
    #print(len(wv))
    if len(wv)>0:
        for j in range(len(wv)):
            if j==0:
                x = (tf.no[bout][:wv[j],0]) 
                fwvs.append(x)
                bout_fwvs.append(bout)

            else:
                x = (tf.no[bout][wv[j-1]:wv[j],0])
                fwvs.append(x)
                bout_fwvs.append(bout)
            #print('Number of bouts for', nickname,'is',len(tf.fr[:]))
assert len(fwvs) == len(bout_fwvs), 'length mismatch'                     
# Create an empty list
#L3 where the animal can end up after 3 decisions max number of node is 14(<15)
# Output should be [0 2 6 13] for normal animals
print(nickname, '[string ending at node 116], bout number')
L3_fwvs = []

# # go through each element in arr
for i in range(len(fwvs)):
    temp = np.array([x for x in fwvs[i] if x <res])
    if len(temp)>0:
        mask = np.insert(np.diff(temp[:]).astype(np.bool),0,True) # mask to remove consecutive reps 
        temp= temp[mask] # apply mask
        L3_fwvs.append(temp)
#         print(temp[:10])
        print(temp, bout_fwvs[i])


L286 [string ending at node 116], bout number
[ 0  2  5 11 24 11 23 11 24 11 23 11  5 12 25 12 26 12 25 12 26 12 25 12
 26 12 25 12 26 12 26 12 25 12 26 12 25 12 26 12 25 12 26 12 25 12 26 12
 25 12 26 12 25 12 26 12 25 12 26 12 26 12  5 11 24 11 23 11 24 11 23 11
 24 11 23 11  5 12 26 12  5 11 23 11  5 12 26 12 25 12 26 12 25 12 26 12
  5 11 23 11  5 12 26 12  5 11 23 11 24 11 23 11  5 12 25 12 26 12 25 12
 26 12 25 12 26 12  5 11 23 11 24 11 23 11 24 11 23 11 24 11 23 11 24 11
 23 11 24 11 23 11 24 11 23 11 24 11 23 11 24 11 23 11  5  2  6 14 30 14
 30 14 29 14 30 14 29 14 30 14 29 14 30 14 29 14 30 14 29 14 30 14 29 14
 30 14 29 14 30 14 29 14 30 14 29 14 30 14 29 14 30 14 30 14 30 14 29 14
 30 14 29 14 29 14 30 14 30 14 30 14  6 14  6 13 28] 0
[28 13 27 13 28] 0
[28] 0
[28 13 27 13 28] 0
[28] 0
[28 13 27 13 28 13 27 13 28] 0
[28] 0
[28 13 27 13 28] 0
[28 13 27 13 28] 0
[28 13 27 13  6 14 29 14 30 14 30 14 30 14  6 13 27 13 28] 0
[28 13 27 13 28 13 27 13 28] 0
[28 13 27 13 28] 0
[28